In [1]:
import numpy as np
import pandas as pd
import scipy
import sys
import os
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss, accuracy_score
from sklearn.preprocessing import StandardScaler
from keras.utils import to_categorical
from collections import Counter
import matplotlib.pyplot as plt
from matplotlib import cm,colors

In [2]:
from lspin import Model
from lspin import convertToOneHot, DataSet_meta

Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
dir_path = os.path.abspath('')
inhibitory = dir_path + '/data/dataframe/mouse/inhibitory_transgenic_data.csv'
excitatory = dir_path + '/data/dataframe/mouse/excitatory_transgenic_data.csv'
data_inhibitory = pd.read_csv(inhibitory)
data_excitatory = pd.read_csv(excitatory)
results_mouse = dir_path + '/results/MLP/transgenic/mouse'

In [4]:
def preprocess_data(df: pd.DataFrame, n_classes: int) -> tuple:
    """
    :param df: raw dataframe to be processed.
    :param n_classes: number of classes in the dataframe.
    :return: split into train, validation and test.
    """
    db = df.dropna(axis=1, how='all')
    db = db.dropna(axis=0)
    irrelevant_columns = ['dendrite_type', 'layer', 'mean_clipped', 'file_name', 'mean_threshold_index',                           'mean_peak_index', 'mean_trough_index', 'mean_upstroke_index',                                           'mean_downstroke_index', 'mean_fast_trough_index']
    db = db.drop([x for x in irrelevant_columns if x in df.columns], axis=1)
    db['transgenic_line'] = pd.Categorical(db['transgenic_line'])
    class_names = dict(enumerate(db['transgenic_line'].cat.categories))
    db['transgenic_line'] = db['transgenic_line'].cat.codes
    scaler = StandardScaler()
    y = db.pop('transgenic_line')
    y = y.values.astype(np.float32)
    y = to_categorical(y, num_classes=n_classes)
    x = db.values.astype(np.float32)
    x = scaler.fit_transform(x)
    x_train, x_val, y_train, y_val = train_test_split(x, y, train_size=0.9, random_state=42)
    x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, train_size=0.65, random_state=42)
    return x_train, y_train, x_val, y_val, x_test, y_test

In [5]:
x_train, y_train, x_val, y_val, x_test, y_test = preprocess_data(data_inhibitory, 5)

In [6]:
data = DataSet_meta(**{'_data':x_train, '_labels':y_train,'_meta':y_train,
                       '_valid_data':x_val, '_valid_labels':y_val,'_valid_meta':y_val,
                       '_test_data':x_test, '_test_labels':y_test,'_test_meta':y_test})

In [7]:
x_train.shape

(356, 20)

In [8]:
y_train.shape

(356, 5)

In [9]:
x_val.shape

(62, 20)

In [10]:
y_val.shape

(62, 5)

In [11]:
x_test.shape

(193, 20)

In [12]:
y_test.shape

(193, 5)

In [13]:
y_train[8]

array([0., 0., 1., 0., 0.], dtype=float32)

## Run LSPIN

In [14]:
# hyper-parameter specification
model_params = {"input_node": x_train.shape[1], "hidden_layers_node": [128, 64, 32, 16], "output_node": 5,
                "feature_selection": True, "gating_net_hidden_layers_node": [10], "display_step": 1000,
                'activation_pred': 'l_relu', 'activation_gating': 'tanh'}


In [15]:
training_params = {'batch_size':x_train.shape[0]}

In [16]:
# objective function for optuna hyper-parameter optimization
def objective(trial):
    global model
    
    # hyper-parameter to optimize: lambda, learning rate, number of epochs
    model_params['lam'] = trial.suggest_loguniform('lam', 1e-3, 1e-2)
    training_params['lr'] = trial.suggest_loguniform('learning_rate', 1e-2, 2e-1)
    training_params['num_epoch'] = trial.suggest_categorical('num_epoch', [2000, 5000, 10000, 15000])

    # specify the model with these parameters and train the model
    model = Model(**model_params)
    train_acces, train_losses, val_acces, val_losses = model.train(dataset=data, **training_params)

    print("In trial:---------------------")
    y_pred = model.test(x_val)
    true = np.argmax(y_val, axis=1)
    accuracy = accuracy_score(true, y_pred)
    return accuracy
        
def callback(study, trial):
    global best_model
    if study.best_trial == trial:
        best_model = model

In [17]:
# optimize the model via Optuna and obtain the best model with smallest validation mse
best_model = None
model = None
study = optuna.create_study(study_name='lspin', direction='maximize')
study.optimize(func=objective, n_trials=10, n_jobs=1, callbacks=[callback])

[I 2022-08-21 17:42:51,194] A new study created in memory with name: lspin


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



C:\MSc\thesis\Code\lspin.py:314: UserWarning: `tf.layers.batch_normalization` is deprecated and will be removed in a future version. Please use `tf.keras.layers.BatchNormalization` instead. In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.BatchNormalization` documentation).
  prev_x = tf.layers.batch_normalization(prev_x, training=is_train)
C:\MSc\thesis\Code\lspin.py:345: UserWarning: `tf.layers.batch_normalization` is deprecated and will be removed in a future version. Please use `tf.keras.layers.BatchNormalization` instead. In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.BatchNormalization` documentation).
  prev_x = tf.layers.batch_normalization(prev_x, training=is_train)


num_samples : 356
Epoch: 1000 train loss=0.093911693 valid loss= 1.394084215 valid acc= 0.629032254


[I 2022-08-21 17:43:03,224] Trial 0 finished with value: 0.7096774193548387 and parameters: {'lam': 0.0012934518594018374, 'learning_rate': 0.08294597552393042, 'num_epoch': 2000}. Best is trial 0 with value: 0.7096774193548387.


...

[I 2022-08-21 17:43:25,099] Trial 2 finished with value: 0.7258064516129032 and parameters: {'lam': 0.009161125671220667, 'learning_rate': 0.03880933574449226, 'num_epoch': 2000}. Best is trial 1 with value: 0.7580645161290323.


Epoch: 2000 train loss=0.083770342 valid loss= 0.999063849 valid acc= 0.725806475
Optimization Finished!
test loss: 1.578796625137329, test acc: 0.6632124185562134
In trial:---------------------


C:\MSc\thesis\Code\lspin.py:314: UserWarning: `tf.layers.batch_normalization` is deprecated and will be removed in a future version. Please use `tf.keras.layers.BatchNormalization` instead. In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.BatchNormalization` documentation).
  prev_x = tf.layers.batch_normalization(prev_x, training=is_train)
C:\MSc\thesis\Code\lspin.py:345: UserWarning: `tf.layers.batch_normalization` is deprecated and will be removed in a future version. Please use `tf.keras.layers.BatchNormalization` instead. In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.BatchNormalization` documentation).
  prev_x = tf.layers.batch_normalization(prev_x, training=is_train)


num_samples : 356
Epoch: 1000 train loss=0.639404714 valid loss= 0.679481626 valid acc= 0.790322602
Epoch: 2000 train loss=0.171460435 valid loss= 0.778588474 valid acc= 0.741935492


KeyboardInterrupt: 

In [ ]:
# get the training gate matrix
gate_mat_train = best_model.get_prob_alpha(x_train)

In [ ]:
best_lr = study.best_params['learning_rate']
best_epoch = study.best_params['num_epoch']
best_lam = study.best_params['lam']

In [ ]:
# test the best model
y_pred = best_model.test(x_test)
            
print("Trial Finished*************")
print("Best model's lambda: {}".format(best_lam))
print("Best model's learning rate: {}".format(best_lr))
print("Best model's num of epochs: {}".format(best_epoch))
print("Test accuracy : {}".format(accuracy_score(np.argmax(y_test, axis=1), y_pred)))

In [ ]:
print(y_pred[1])
np.argmax(y_test[1])

### Comparing the training gates to the ground truth

In [ ]:
# cmap = cm.Blues
# bounds=[0,0.5,1]
# norm = colors.BoundaryNorm(bounds, cmap.N)
#
# title_size = 30
# xtick_size = 20
# ytick_size = 20
# xlabel_size = 35
# ylabel_size = 35
# colorbar_tick_size = 20
# title_pad = 10

In [ ]:
# fig, axes = plt.subplots(1, 2,sharex=False, sharey=True,figsize=(8, 6))
#
# sorted_order = np.concatenate((np.where(train_label == 1)[0],np.where(train_label == 2)[0]))
#
# im1 = axes[0].imshow(ref_feat_mat_train[sorted_order,:].astype(int),aspect='auto',cmap=cmap, norm=norm)
# axes[0].set_title("Ground Truth",fontsize=title_size,fontweight="bold",pad=title_pad)
# axes[0].set_ylabel("Sample Index",fontsize=ylabel_size)
# axes[0].set_yticks([1,3,5,7,9])
# axes[0].set_yticklabels([2,4,6,8,10],fontsize=ytick_size)
# axes[0].set_xticks(list(range(5)))
# axes[0].set_xticklabels(list(range(1,6)),fontsize=xtick_size)
# axes[0].set_xlabel("Feature Index",fontsize=xlabel_size,labelpad=-5)
#
# cbar = fig.colorbar(im1,ax=axes[0], cmap=cmap, norm=norm, boundaries=bounds, ticks=[0, 1])
# cbar.ax.tick_params(labelsize=colorbar_tick_size)
#
# im2 = axes[1].imshow(gate_mat_train[sorted_order,:],aspect='auto',cmap=cmap)
# axes[1].set_title("LLSPIN Gates",fontsize=title_size,fontweight="bold",pad=title_pad)
# axes[1].set_yticks([1,3,5,7,9])
# axes[1].set_yticklabels([2,4,6,8,10],fontsize=ytick_size)
# axes[1].set_xticks(list(range(5)))
# axes[1].set_xticklabels(list(range(1,6)),fontsize=xtick_size)
# axes[1].set_xlabel("Feature Index",fontsize=xlabel_size,labelpad=-5)
#
# cbar = fig.colorbar(im2,ax=axes[1])
# cbar.ax.tick_params(labelsize=colorbar_tick_size)
#
# plt.tight_layout()